In [8]:
'''
Function:Load the data base and query
Date: 09/20/2017
Author: Yu (Brian) Yao

Input: 
    long: Longitude coordinate in float.
    lat:  Latitude coordinate in float.
    
Output:
    Top three nearest road. 
    
Database file:
    0 - 6:   id, highway, footway, name, tiger:cfcc, lanes, service
    7 - 12:  surface, oneway, amenity, maxspeed, crossing, lit
    13 - 16: lanes:backward, lanes:forward, source, parking
    17 - 19: sidewalk, turn:lanes:backward, turn:lanes:forward
    20 - 24: turn:lanes, destination:ref, barrier, tunnel, water
    25 - 26: hourse, railway
    27:      geometry
    *28:     distance 
    
'''
import sqlite3
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import folium
%matplotlib tk

MAX = 1E-4
MIN = 1E-5
class sql():
    def __init__(self, db_name):
        # connect to database
        self.conn = sqlite3.connect(db_name)
        self.conn.enable_load_extension(True)
        self.conn.load_extension("mod_spatialite")
        
        self.num_roads = 5
        
    def load_tunnel(self):
#         res = self.conn.execute("SELECT tunnel, COUNT(tunnel)\
#                                  FROM ('import.osm_roads') as rd\
#                                  GROUP By tunnel
#                                  ORDER By number DESC")
        res = self.conn.execute("SELECT tunnel, ST_AsText(ST_Transform(rd.geometry,4326))\
                                 FROM ('import.osm_roads') as rd")
        tunnels = res.fetchall()
        print (tunnels[0])
        tunls = []
        geos = []
        for i in range(len(tunnels)):
            tunls.append(tunnels[i][0])
            geos.append(tunnels[i][1])
        tunnel_types = {}
        tunnel_types['tunnel'] = tunls
        tunnel_types['geometry'] = geos
        
        return tunnel_types
        
    def count_highway(self):
        res = self.conn.execute("SELECT highway, COUNT(highway) as number \
                                 FROM ('import.osm_roads') as rd \
                                 GROUP By highway \
                                 ORDER By number DESC")
        highways = res.fetchall()
        res = self.conn.execute("SELECT highway \
                                 FROM ('import.osm_roads') as rd")
        highways = res.fetchall()
        
        fields = []
        for i in range(len(highways)):
            fields.append(highways[i][0])        
        sns.set(style="darkgrid")
        
        road_types = {}
        road_types['class']=fields
        
        
        f = plt.figure(figsize=(15,15.4),dpi=80)
        ax = f.add_subplot(111)
        ax = sns.countplot(y='class', data = road_types, ax=ax)
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 1.1, box.height])
        ax.set_yticklabels(ax.get_yticklabels(), fontsize=16)
#         ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)
        
    def count_surfaces(self):
        
        res = self.conn.execute("SELECT COUNT(*) \
                                 FROM ('import.osm_roads') as rd \
                                 WHERE rd.surface='unpaved' OR \
                                       rd.surface='dirt' OR \
                                       rd.surface='gravel' OR \
                                       rd.surface='ground' OR \
                                       rd.surface='dirt;unpaved' OR \
                                       rd.surface='compacted' OR \
                                       rd.surface='earth' OR \
                                       rd.surface='mud' OR \
                                       rd.surface='sand' OR \
                                       rd.surface='woodchips' OR \
                                       rd.surface='grass' OR \
                                       rd.surface='fine_gravel'")
        unpaved_roads = res.fetchall()
        res = self.conn.execute("SELECT COUNT(*) \
                                 FROM ('import.osm_roads') as rd \
                                 WHERE rd.surface='paved' OR \
                                       rd.surface='asphalt' OR \
                                       rd.surface='concrete' OR \
                                       rd.surface='sett' OR \
                                       rd.surface='concrete:lanes' OR \
                                       rd.surface='concrete:plates' OR \
                                       rd.surface='metal' OR \
                                       rd.surface='wood' OR \
                                       rd.surface='paving_stones' OR \
                                       rd.surface='cobblestone' OR \
                                       rd.surface='pebblestone' OR \
                                       rd.surface='brick'")
        paved_roads = res.fetchall()
        res = self.conn.execute("SELECT COUNT(*) \
                                 FROM ('import.osm_roads') as rd \
                                 WHERE rd.surface=''")
        unlabelled = res.fetchall()
        res = self.conn.execute("SELECT COUNT(*) \
                                 FROM ('import.osm_roads') as rd")
        total = res.fetchall()
        print ('# of unpaved roads: ', unpaved_roads[0][0])
        print ('# of paved roads: ', paved_roads[0][0])
        print ('# of unlabeled roads: ', unlabelled[0][0])
        print ('Total # of roads: ', total[0][0])
        
        
        
        # plot coutplot 
        res = self.conn.execute("SELECT rd.surface \
                                 FROM ('import.osm_roads') as rd")
        surface_types = res.fetchall()
        concrete_roads = ['paved', 'asphalt', 'concrete', 'metal', 'wood']
        stone_roads = ['paving_stones', 'cobblestone', 'pebblestone', 'brick', 'sett', 
                       'concrete:lanes', 'concrete:plates', 'paving_stones']
        unpaved_roads = ['unpaved', 'dirt', 'gravel', 'ground', 'dirt;unpaved', 'compacted', 
                         'earth', 'mud', 'sand', 'woodchips', 'grass', 'fine_gravel']
        fields = []
        for i in range(len(surface_types)):
            surface = surface_types[i][0]
            if surface in concrete_roads:
                fields.append('concrete roads')
            elif surface in stone_roads:
                fields.append('stone roads')
            elif surface in unpaved_roads:
                fields.append('unpaved roads')
            else:
                fields.append('unlabelled roads')
        surface_types = {}
        surface_types['types']=fields
        
        sns.set(style="darkgrid")
        f = plt.figure(figsize=(16,8.4),dpi=80)
        ax = f.add_subplot(111)
        ax = sns.countplot(y='types', data = surface_types, ax=ax)
#         box = ax.get_position()
#         ax.set_position([box.x0, box.y0, box.width * 1.1, box.height])
#         ax.set_xticklabels(ax.get_xticklabels(), fontsize=12)
    
        # add numbers
        for p in ax.patches:
            ax.text(p.get_width() + 10000,
                    p.get_y()+p.get_height()*0.3,
                    '{:1.5f}'.format(p.get_width()/total[0][0]),
                    ha="center", rotation=-90)
        ax.set_yticklabels(ax.get_yticklabels(), fontsize=16)

    def reorder_roads(self, location):
        '''
        Reorder road data by distance 
        '''
        long = location[0]
        lat = location[1]
        '''Find the nearest road''' 
        # data = conn.execute("SELECT *, MIN(DISTANCE(geometry, ST_POINT( -83.702175,42.297199))) as dist \
        #                     FROM OSM_AA_POLYLINES_NEW as osm \
        #                     JOIN ways_tags as wt USING (id)")

        '''Order the roads by distances to the point.'''
        res = self.conn.execute("SELECT *,ST_Distance(osm.geometry, ST_POINT(?,?)) as dist\
                            FROM OSM_AA_POLYLINES_NEW as osm\
                            ORDER BY ST_Distance(osm.geometry, ST_POINT(?,?))",(long,lat,long,lat))
        data = res.fetchall()
        nearest_road = self.check_roads(data)
        
        res = self.conn.execute("SELECT AsText(?), NumPoints(?)",(nearest_road['geometry'], nearest_road['geometry']))
        data = res.fetchall()
        
        return nearest_road
    
    def check_roads(self, data):
        '''
        Given the reordered road data, find the nearest possible road
        '''
        flag = True
        i = 0
        while flag:
            road = self.load_dict(data[i])
            if i == 0 :
                nearest_road = road
            else:
                if road['dist'] - dist_prev < MIN:
                    # Both roads are close
                    if highway_prev == 'footway':
                        # if the closer road is a footway, it may be false
                        # check the next close road
                        if road['highway'] != 'footway':
                            nearest_road = road
                        continue
                elif road['dist'] - dist_prev > MAX:
                    # new road is far
                    return  nearest_road
            dist_prev = road['dist']
            highway_prev = road['highway']
            i += 1
        return nearest_road
            
    def load_dict(self, data):
        road = {}
        road['id'] = data[0]
        road['highway'] = data[1]
        road['footway'] = data[2]
        road['name'] = data[3]
        road['cfcc'] = data[4]
        road['lanes'] = data[5]
        road['service'] = data[6]
        road['surface'] = data[7]
        road['oneway']= data[8]
        road['amenity'] = data[9]
        if data[10] != None:
            road['maxspeed'] = float(data[10][0:2])*0.44704 # mph to m/s
        road['crossing'] = data[11]
        road['lit'] = data[12]
        road['lanes_back'] = data[13]
        road['lanes_forward'] = data[14]
        road['source'] = data[15]
        road['parking'] = data[16]
        road['sidewald'] = data[17]
        road['turn_back'] = data[18]
        road['turn_forward'] = data[19]
        road['turn'] = data[20]
        road['destination'] = data[21]
        road['barrier'] = data[22]
        road['tunnel'] = data[23]
        road['water'] = data[24]
        road['horse'] = data[25]
        road['railway'] = data[26]
        road['geometry'] = data[27]
        road['dist'] = data[28]
        return road
def string2points(road, tunnel, num, m):

    i=11
    coord=''
    points = []
    while road[i]:
        if road[i] == ')':
            lat = float(coord)
            points.append((lat,long))
            break
        if road[i] == ',':
            lat = float(coord)
            points.append((lat,long))
            coord = ''
            i = i+2
            continue
        if road[i]==' ':
            long = float(coord)
            coord = ''
            i = i+1
            continue
        coord = coord+road[i]
        i = i+1

    if tunnel == 'yes':
        color = 'red'
    else:
        color = 'green'
    folium.PolyLine(points, color=color, weight=2, opacity=1).add_to(m)
    print ('# of iter: ', num)
    return points
    
    
# SELECT id, source, wn.node_id,
# MIN(DISTANCE(osm.geometry, ST_POINT( -83.702175,42.297199))) as dist 
# FROM OSM_AA_POLYLINES_NEW AS osm,
# ways_nodes AS wn
# WHERE osm.id=wn.way_id

'''Find the number of points and the points coordinate of a road.'''
# SELECT AsText(geometry), NumPoints(geometry),X(PointN(geometry,2))
# FROM OSM_AA_POLYLINES_NEW as osm
# WHERE osm.id='4411729'

'Find the number of points and the points coordinate of a road.'

In [3]:
db_name = "data/AL_osm.db"
SQL = sql(db_name)
#tunnel_types = SQL.load_tunnel()
#SQL.count_surfaces()
#SQL.count_highway()

In [86]:
if __name__ == "__main__":
    db_name = "data/OSM_AA.osm.db"
    SQL = sql(db_name)
    
    long = -83.702175
    lat = 42.297199
    location = [long, lat]
    nearest_road = SQL.reorder_roads(location)
    print (nearest_road['highway'], nearest_road['maxspeed'])

residential 13.4112


In [2]:
from joblib import Parallel, delayed
import multiprocessing
import folium

num_cores = multiprocessing.cpu_count()

m = folium.Map(location=[43.5, -84.5], zoom_start=7, tiles="cartodbpositron")
             
# plot each polylines in a loop, very slow and memory comsuming
# for i, road in enumerate(tunnel_types['geometry']):
#     string2points(road, tunnel_types['tunnel'][i], i, m)
#     if i >= 5000:
#         break
# Parallel(n_jobs=num_cores)\
#         (delayed(string2points)(road, tunnel_types['tunnel'][i], i, m)\
#         for i, road in enumerate(tunnel_types['geometry']))

# for i, road in enumerate(tunnel_types['geometry']):
#     points = SQL.string2points(road, tunnel_types['tunnel'][i])
#     print (i)


In [4]:
# count the number of tunnels in each county
query  = """
        SELECT SUM(case T when '' then 0 else 1 end) as num_tunnel, N as county_name, Geom as geometry
        FROM
        (
            SELECT tunnel as T, name as N, Hex(ST_AsBinary(SetSRID(ctgeometry, 4326))) as Geom
            FROM
            (
                SELECT rd.osm_id as osm_id, rd.geometry as geometry, rd.tunnel as tunnel, county.name as name, county.geometry as ctgeometry
                FROM "import.osm_roads" as rd,
                     cb_2016_us_county_500k as county
                WHERE rd.ROWID IN (
                    SELECT ROWID
                    FROM SpatialIndex
                    WHERE f_table_name = 'import.osm_roads' AND f_geometry_column = 'GEOMETRY' AND
                      search_frame = ST_Transform(ST_Envelope(SetSRID(county.geometry, 4326)), 3857))
            )
            WHERE within(ST_PointN(ST_Transform(geometry,4326),1), ctgeometry)
        )
        GROUP By N
        """
# query = """
#     SELECT sum(num) as num_tunnel, name as county_name, Geom as geometry
#     FROM 
#     (
#         SELECT COUNT(*) as num, N as name, Geom
#         FROM
#         (
#         SELECT tunnel as T, name as N, Hex(ST_AsBinary(SetSRID(ctgeometry, 4326))) as Geom
#         FROM
#         (
#             SELECT rd.osm_id as osm_id, rd.geometry as geometry, rd.tunnel as tunnel, county.name as name, county.geometry as ctgeometry
#             FROM "import.osm_roads" as rd,
#                 cb_2016_us_county_500k as county
#                 WHERE rd.ROWID IN (
#                     SELECT ROWID
#                     FROM SpatialIndex
#                     WHERE f_table_name = 'import.osm_roads' AND f_geometry_column = 'GEOMETRY' AND
#                         search_frame = ST_Transform(ST_Envelope(SetSRID(county.geometry, 4326)), 3857))
#         )
#         WHERE within(ST_PointN(ST_Transform(geometry,4326),1), ctgeometry)
#         )
#         Where T!=''
#         GROUP By N
#         UNION ALL
#         SELECT 0 as num, county.name as name, Hex(ST_AsBinary(SetSRID(geometry,4326))) as Geom
#         FROM cb_2016_us_county_500k as county
#     )t
#     GROUP By name
#     """
# query = """
#         SELECT county.name, ST_Area(county.geometry) as area, Hex(ST_AsBinary(SetSRID(county.geometry, 4326))) as geometry
#         FROM cb_2016_us_county_500k as county
#         """
res = SQL.conn.execute(query)
data = res.fetchall()
data[0]

(0,
 u'Alcona',
 u'010600000001000000010300000001000000930000003BC1FEEBDCF854C0E3326E6AA06D46404C512E8D5FF354C0874B8E3BA56D464060014C1938F354C0D61C2098A36D4640B24AE9995EF254C03DCF9F36AA6D46408A3DB48F15F154C0562B137EA96D4640A2258FA7E5E654C08429CAA5F16D4640DDCF29C8CFE654C0C4B0C398F46D46407D02284696E554C0FFAECF9CF56D46400DE02D90A0E354C00D164ED2FC6D464096CCB1BCABE154C07A51BB5F056E4640E257ACE122DB54C0C35F9335EA6D4640F4FA93F8DCD954C0810871E5EC6D4640AF08FEB792D454C04485EAE6E26D464010E4A08499D454C0658C0FB3976D4640994869368FD454C0895DDBDB2D6D464085B35BCB64D454C03F1D8F19A86C4640F6B2EDB435D454C01F80D4264E6C46407EFACF9A1FD454C0C0046EDDCD6B46401F4AB4E4F1D354C08507CDAE7B6B464094675E0EBBD354C0D2FC31AD4D6B4640A31CCC26C0D354C0B5A338471D6B4640BB07E8BE9CD354C031CC09DAE46A464022C154336BD354C0BDE0D39CBC6A4640B9A981E673D354C0E4637781926A464014E63DCE34D354C0779E78CE166A4640C45BE7DF2ED354C002EFE4D363694640B5DD04DF34D354C056F31C91EF6846406954E0641BD354C0BB96900F7A684640C579AE42EDD254C03CD893A4E7674640EA059FE6E

In [5]:
import geopandas as gpd
from shapely import wkb
from binascii import unhexlify
from codecs import encode
import pysal

def as_geom(hex):
    return wkb.loads(unhexlify(encode(hex, "hex")))

#county_map = gpd.pd.read_sql(query, SQL.conn)
county_map = gpd.GeoDataFrame.from_postgis(query, SQL.conn, geom_col='geometry',crs={"init":"epsg:4326"})

#county_map["geometry"] =county_map["geometry"].apply(as_geom)
#wkt.loads(county_map["geometry"][0])
#a = wkt.loads('POLYGON((-92.411995 41.509548, -92.355389 41.509646, -92.345287 41.509677, -92.297494 41.50979, -92.287642 41.509828, -92.224604 41.509901, -92.200892 41.510019, -92.181784 41.510038, -92.156622 41.510156, -92.081916 41.510295, -92.078941 41.510302, -92.07644 41.510293, -92.071452 41.510371, -92.065058 41.510403, -92.051674 41.510437, -92.04251 41.510412, -92.039328 41.510635, -92.037284 41.510482, -91.946043 41.510749, -91.946144 41.46106, -91.946235 41.453166, -91.946643 41.424634, -91.946448 41.410126, -91.946612 41.381348, -91.946697 41.370201, -91.946713 41.366589, -91.94669 41.364258, -91.946745 41.359367, -91.946815 41.352078, -91.946935 41.33758, -91.945091 41.251386, -91.944762 41.240378, -91.944743 41.236265, -91.944641 41.222029, -91.945571 41.163578, -91.953026 41.163667, -91.964754 41.16376, -92.022733 41.163548, -92.063336 41.163194, -92.179974 41.162662, -92.227955 41.162539, -92.295933 41.162303, -92.320012 41.16224, -92.334238 41.162155, -92.406547 41.161922, -92.410233 41.161942, -92.410289 41.16918, -92.41007 41.176734, -92.410342 41.1844, -92.410382 41.190896, -92.410392 41.20351, -92.410592 41.205385, -92.410493 41.24989, -92.410775 41.267301, -92.410828 41.271544, -92.410893 41.278753, -92.411155 41.336124, -92.411277 41.350586, -92.411327 41.368688, -92.411569 41.408481, -92.411673 41.423212, -92.412008 41.484792, -92.411995 41.509548))')
#county_map.set_geometry("geometry", crs={"init":"epsg:4326"})
#county_map.plot(column='area', categorical=True, figsize=(14, 8))
type(county_map)
#county_map.plot(column='num_tunnel',scheme='fisher_jenks', cmap=plt.cm.Blues,
#                figsize=(14, 8), k=7, legend=True)
county_map.head()


,num_tunnel,county_name,geometry
0,0,Alcona,"(POLYGON ((-83.88848399999999 44.856458, -83.8..."
1,0,Alger,"(POLYGON ((-86.70904 46.54521, -86.704854 46.5..."
2,1,Allegan,"(POLYGON ((-86.2700774930095 42.4269033510126,..."
3,0,Alpena,"(POLYGON ((-83.217371 45.05406199999999, -83.2..."
4,0,Antrim,(POLYGON ((-85.43805999999999 44.8651379999999...


In [6]:
# make the colormap
from branca.colormap import linear
colormap = linear.YlGn.scale(county_map['num_tunnel'].min(), county_map['num_tunnel'].max())
print(colormap(1))
colormap

#fdfec9


In [7]:
# convert the table to a dictionary to map a feature to it's number of tunnels
county_map_dict = county_map.set_index('county_name')['num_tunnel']
county_map_dict['Washtenaw']

41

In [120]:

m = folium.Map(location=[42.5, -83.5], zoom_start=7, tiles="cartodbpositron")
# m.choropleth(
#     geo_data=county_map,
#     data=county_map,
#     columns=['county_name','num_tunnel'],
#     key_on='feature.id',
#     legend_name='Tunnels (#)', 
#     fill_color='YlGn',
#     fill_opacity=0.4,
#     highlight=True)

folium.GeoJson(county_map,
               name='num_tunnel',
               style_function=lambda feature: {
                   'fillColor': colormap(county_map_dict[feature['properties']['county_name']]),
                   'color': 'black',
                   'weight': 1,
                   'dashArray': '5, 5',
                   'fillOpacity': 0.9,
                }).add_to(m)
# add legend
colormap.caption = '# of Tunnels'
colormap.add_to(m)

m

In [1]:
# use new database file to plot
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import folium
import geopandas as gpd
from shapely import wkb
from binascii import unhexlify
from codecs import encode
import pysal
%matplotlib tk

db_name = "data/USA_osm.db"

conn = sqlite3.connect(db_name)
conn.enable_load_extension(True)
conn.load_extension("mod_spatialite")

query = """
        SELECT db.ROWID, db.num_tunnel as num_tunnel, db.county_name as county_name, Hex(ST_AsBinary(db.geometry)) as geometry
        FROM county_tunnel as db
        """
county_map = gpd.GeoDataFrame.from_postgis(query, conn, geom_col='geometry',crs={"init":"epsg:4326"})
county_map.head


DatabaseError: Execution failed on sql '
        SELECT db.ROWID, db.num_tunnel as num_tunnel, db.county_name as county_name, Hex(ST_AsBinary(db.geometry)) as geometry
        FROM county_tunnel as db
        ': database disk image is malformed

In [23]:
# plot the whole tunnel distribution in USA
# make the colormap
from branca.colormap import linear,LinearColormap
import math

n=len(county_map)
colormap = linear.Hot.scale(county_map['num_tunnel'].min(), county_map['num_tunnel'].max())
value_list = range(county_map['num_tunnel'].min()+1, county_map['num_tunnel'].max()+1)
# add a log10 index, and change the color map to stepcolormap
max_ = max(value_list)
min_ = min(value_list)
index = [5**(math.log10(min_) + i * (math.log10(max_) -math.log10(min_)) * 1./n) for i in range(1+n)]
log_colormap = colormap.to_step(index=index, round_method='int')
log_colormap

yes
[1.0, 1.000938210420035, 1.0018773010788626, 1.0028172728023315, 1.0037581264170659, 1.0046998627504653, 1.0056424826307058, 1.00658598688674, 1.0075303763482983, 1.0084756518458904, 1.0094218142108038, 1.0103688642751072, 1.0113168028716493, 1.01226563083406, 1.0132153489967521, 1.0141659581949203, 1.0151174592645438, 1.0160698530423855, 1.0170231403659933, 1.0179773220737016, 1.0189323990046306, 1.0198883719986882, 1.02084524189657, 1.0218030095397608, 1.0227616757705342, 1.0237212414319548, 1.0246817073678776, 1.0256430744229497, 1.0266053434426101, 1.0275685152730918, 1.028532590761421, 1.029497570755419, 1.0304634561037027, 1.0314302476556847, 1.0323979462615747, 1.03336655277238, 1.034336068039907, 1.0353064929167604, 1.036277828256345, 1.0372500749128664, 1.038223233741332, 1.0391973055975505, 1.0401722913381346, 1.0411481918205, 1.0421250079028668, 1.0431027404442605, 1.0440813903045125, 1.045060958344261, 1.0460414454249516, 1.0470228524088379, 1.0480051801589827, 1.048988

In [24]:
# convert the table to a dictionary to map a feature to it's number of tunnels
county_map_dict = county_map.set_index('rowid')['num_tunnel']

In [25]:
m = folium.Map(location=[42.5, -83.5], zoom_start=7, tiles="cartodbpositron")
folium.GeoJson(county_map,
               name='num_tunnel',
               style_function=lambda feature: {
                   'fillColor': log_colormap(county_map_dict[feature['properties']['rowid']]+1),
                   #'fillColor': colors.to_hex(colormap.to_rgba(county_map_dict[feature['properties']['rowid']]+1)),
                   #'color': 'black',
                   'weight': 1,
                   'dashArray': '5, 5',
                   'fillOpacity': 0.9,
                }
              ).add_to(m)
# add legend
log_colormap.caption = '# of Tunnels'
log_colormap.add_to(m)
folium.LayerControl().add_to(m)
m.save('output.html')


In [87]:
# ----------------------------------------- colormap in matplotlib ----------------------------------------------
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.colors as colors
# import matplotlib.cm as cm
# from matplotlib.mlab import bivariate_normal
# print county_map['num_tunnel'].min()
# print county_map['num_tunnel'].max()
    
# norm = colors.LogNorm(vmin=county_map['num_tunnel'].min()+1, vmax=county_map['num_tunnel'].max())
# cmap = cm.coolwarm
# colormap = cm.ScalarMappable(norm=norm, cmap=cmap)

0
1076


u'#b70d28'